In [74]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import plotly.express as px
import plotly.subplots as sp

In [75]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSlevels_df_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']

In [76]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url, dtype=None, parse_dates=None):
    req = urllib.request.Request(url)
    req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req), dtype=dtype, parse_dates=parse_dates)

ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all", dtype=dtypes)
ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life', 'electric_quadrupole']]
ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
ground_states_df = ground_states_df.copy()
ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
ground_states_df = ground_states_df.set_index(['z', 'a'], drop=True)

## Load saved data

In [77]:
levels_df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


levels_df.loc[levels_df['half_life'] == 'STABLE', 'half_life'] = 'inf'
levels_df['half_life'] = levels_df['half_life'].astype('Float64')

levels_df['a'] = levels_df['z'] + levels_df['n']

levels_df = levels_df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
levels_df['energy'] = levels_df['energy'] / 1000 #MeV
levels_df['ripl_shift'] = levels_df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

levels_df = levels_df.set_index(['z', 'a'], drop=True)

In [78]:
# Filter out unknown energies

levels_df = levels_df[levels_df['energy_shift'].isna()]
levels_df = levels_df.drop(['energy_shift', 'ripl_shift'], axis='columns')

In [79]:
# Keep only even-even nuclei

df = levels_df[(levels_df.index.get_level_values('z') % 2 == 0) & (levels_df.index.get_level_values('a') % 2 == 0)]

In [80]:
# Filter out uncertain jp

df = df[df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

df['j'] = df['jp'].str[:-1]
df['p'] = df['jp'].str[-1]

odd_spins = df['j'].str.contains('/')
even_spins = ~odd_spins

df.loc[odd_spins,'j_float'] = df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
df.loc[even_spins,'j_float'] = df[even_spins]['j'].astype(float)

df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
df.loc[even_spins,'j_evenness'] = (df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)

df['p_bit'] = df['p'].str.fullmatch(r'\-').astype(int)

In [81]:
# Only consider first even levels i.e. 0^+_1, 2^+_1, 4^+_1, 6^+_1, 8^+_1, ...

first_even_levels = df[(df['jp_order'] == 1) & df['j_evenness'].fillna(False) & (df['p_bit'] == 0)].copy()

# Number of quanta of quadrupole oscillation
first_even_levels['quanta'] = (first_even_levels['j_float'].astype(int) // 2)

# Remove extraenous information
first_even_levels = first_even_levels.drop(columns=['idx', 'jp', 'jp_order', 'j', 'p', 'j_float', 'j_evenness', 'p_bit'])

In [82]:
# Filter out not enough levels
first_even_levels = first_even_levels.loc[first_even_levels.groupby(level=first_even_levels.index.names).size() >= 4]

In [83]:
def get_osc_energy(n, energy_quantum, shift):
    harmonics = n * energy_quantum
    shifts = n * (n-1) / 2 * shift

    return harmonics + shifts

def fit(group, func):

    x = group['quanta']
    y = group['energy']

    # fit
    popt, pcov = curve_fit(func, x, y)

    # prediction
    y_pred = func(x, *popt)

    # r-squared
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    dof_res = len(x) - 2 - 1
    dof_tot = len(x) - 1
    r2 = 1 - ((ss_res/dof_res) / (ss_tot/dof_tot))

    # results
    results = pd.DataFrame({'energy_quantum': popt[0], 'shift': popt[1], 'r2': r2, 'quanta': x, 'energy_pred': y_pred})
    
    return  results

In [84]:
first_even_levels_groups = first_even_levels.groupby(level=df.index.names, as_index=False)

oscillator_fit = first_even_levels_groups.apply(lambda group: fit(group, get_osc_energy)).droplevel(0)

In [86]:
merged = first_even_levels.merge(oscillator_fit, how='left', on=['z','a', 'quanta'])

oscillators = merged.groupby(level=merged.index.names, as_index=True).head(1)[['symbol', 'n', 'energy_quantum', 'shift', 'r2']]
oscillators['normalized_shift'] = oscillators['shift'] / oscillators['energy_quantum']

def get_best_r2(df, min_r2):
    return df[df['r2'] >= min_r2]

def get_worst_r2(df, max_r2):
    return df[df['r2'] < max_r2]

In [96]:
mean = oscillators['r2'].mean()

percentages = [68, 95, 99.7, 100]

count = oscillators['r2'].count()
stdevs_indices = [int(count * percentage/100) for percentage in percentages]
stdevs_r2 = [oscillators.sort_values(by='r2', ascending=False).iloc[stdevs_idx-1]['r2'] for stdevs_idx in stdevs_indices]
percentages = [stdev_idx / count * 100 for stdev_idx in stdevs_indices]


fig = px.histogram(oscillators, x='r2', nbins=100, title='Distribution of R² Values For Oscillator with Anharmonicity')
fig.update_layout(
    xaxis_title='Coefficient of Determination (R²)',
    yaxis_title='Count',
    width=800,
    showlegend=False
)
# fig.add_vline(x=mean, line_color="black", annotation_text="Mean")


for percentage, stdev_r2 in zip(percentages, stdevs_r2):
    print(f"{percentage:.1f}% have R² >= {stdev_r2:.4f}")
    fig.add_vline(x=stdev_r2, line_dash="dash", line_color="red", annotation_text=f"{percentage:.1f}%")

fig.show()

67.5% have R² >= 0.9916
94.4% have R² >= 0.9091
99.2% have R² >= 0.7235
100.0% have R² >= 0.5591


In [ ]:
get_worst_r2(oscillators, stdevs_r2[1]).sort_values(by='r2', axis=0, ascending=True)

symbol    n        r2
z  a                        
82 208     Pb  126  0.559129
   206     Pb  124  0.723459
28 64      Ni   36  0.796669
40 90      Zr   50  0.815670
26 54      Fe   28  0.899490
60 142     Nd   82  0.901039
8  18       O   10  0.902893

In [ ]:
fig = px.scatter(get_worst_r2(oscillators, stdevs_r2[1]).reset_index(), 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Lowest 5% of Oscillator Fits',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()

In [ ]:
fig = px.scatter(get_best_r2(oscillators, stdevs_r2[1]).reset_index(), 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Highest 95% of Oscillator Fits',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
        #   color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          color_continuous_scale='plasma',
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [ ]:
fig = px.scatter(get_best_r2(oscillators, 0.99).reset_index(), 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Best Oscillator Fits (R² > 0.99)',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
        #   color_continuous_scale=[(0,'#0d0887'), (0.8,'#cc4778'), (1,'#f0f921')],
          color_continuous_scale='plasma',
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

In [ ]:
fig = px.line(get_best_r2(oscillators, 0.5).reset_index(), 
                 x='a', 
                 y='r2', 
                 line_group='z',
                 title='R² Scores vs Weight for Oscillators, Grouped by Isotopes',
                 labels={'a': 'Weight (A)', 
                        'r2': 'Coefficient of Determination (R²)',
                        'z': 'Atomic Number (Z)'})

fig.show()

fig = px.line(get_best_r2(oscillators, 0.5).reset_index(), 
                 x='a', 
                 y='r2', 
                 line_group='n',
                 title='R² Scores vs Weight for Oscillators, Grouped by Isotones',
                 labels={'a': 'Weight (A)', 
                        'r2': 'Coefficient of Determination (R²)',
                        'z': 'Atomic Number (Z)'})

fig.show()

In [ ]:
residuals_df = merged.copy()
residuals_df = residuals_df.reset_index()
residuals_df["name"] = residuals_df["a"].astype(str) + residuals_df["symbol"]
residuals_df["residuals"] = residuals_df["energy"] - residuals_df["energy_pred"]
residuals_df["normalized_residuals"] = residuals_df["residuals"] / residuals_df["energy_quantum"]

best_residuals = get_best_r2(residuals_df, 0.9992)
fig = px.scatter(best_residuals, x="quanta", y="normalized_residuals", facet_col="name", facet_col_wrap=6, 
                 labels={"quanta": "Phonons (n)", "normalized_residuals": "Normalized Residuals", "name": "Nuclide"},
                 height=800)
fig.update_yaxes(range=[-0.7, 0.7])
fig.show()

In [118]:
mean = oscillators['normalized_shift'].mean()
stdev = oscillators['normalized_shift'].std()

fig = px.histogram(oscillators, x='normalized_shift', nbins=100, 
                   title='Distribution of Normalized Shifts')
fig.update_layout(
    xaxis_title='Normalized Shift',
    yaxis_title='Count',
    width=800,
    showlegend=False
)

fig.show()

In [175]:
oscillators_pert = oscillators[oscillators['normalized_shift'].between(-0.5, 0.5)]
oscillators_nonpert = oscillators[~oscillators['normalized_shift'].between(-0.5, 0.5)]

print(f"Perturbative shifts (within 0.5): {oscillators['normalized_shift'].between(-0.5, 0.5).mean()*100:.1f}%")
print("Of remaining: ")

mean = oscillators_pert['normalized_shift'].mean()
stdev = oscillators_pert['normalized_shift'].std()

print(f"Mean: {mean:.3f}")
print(f"Standard Deviation: {stdev:.3f}")


fig = px.histogram(oscillators_pert, x='normalized_shift', nbins=50, 
                   title='Distribution of Normalized Shifts')
fig.update_layout(
    xaxis_title='Normalized Shift of Oscillators with Perturbative Shifts',
    yaxis_title='Count',
    width=800,
    showlegend=False
)

fig.add_vline(x=mean, line_color="black", annotation_text="Mean")
for i in range(1,4):
    # print(f"{percentage:.1f}% have R² >= {stdev_r2:.3f}")
    fig.add_vline(x=mean-i*stdev, line_dash="dash", line_color="red", annotation_text=f"-{i}σ")
    fig.add_vline(x=mean+i*stdev, line_dash="dash", line_color="red", annotation_text=f"+{i}σ")

    percentage = oscillators_pert['normalized_shift'].between(mean-i*stdev, mean+i*stdev).mean()

    print(f"{percentage*100:.1f}% are within {i}σ of mean")   

fig.show()

Perturbative shifts (within 0.5): 83.3%
Of remaining: 
Mean: 0.118
Standard Deviation: 0.208
66.7% are within 1σ of mean
96.2% are within 2σ of mean
100.0% are within 3σ of mean


In [247]:
fig = px.scatter( 
          x=oscillators_nonpert.reset_index()['n'],
          y=oscillators_nonpert.reset_index()['z'], 
          color=oscillators_nonpert.reset_index()['normalized_shift'].abs(),
          title='Normalized Shift for Oscillators with Non-Perturbative Shifts',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'normalized_shift': 'Normalized Shift'},
        #   color_continuous_scale=new_colors,
          color_continuous_scale='Viridis',
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()

In [248]:
fig = px.scatter(oscillators_pert.reset_index(), 
          x='n',
          y='z', 
          color='normalized_shift',
          title='Normalized Shift for Oscillators with Perturbative Shifts',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'normalized_shift': 'Normalized Shift'},
          color_continuous_scale='RdBu',
          color_continuous_midpoint=0,
          height=600,
          width=600)

magic_numbers_n = [2, 8, 20, 50, 82, 126]
magic_numbers_z = [2, 8, 20, 50, 82]

for i, m in enumerate(magic_numbers_n):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

for i, m in enumerate(magic_numbers_z):
    fig.add_hline(y=m, line_dash="dash", line_color="gray")


fig.show()